In [2]:
import argparse
import os
import time
import torch
import yolo

    
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and args.use_cuda else "cpu")
    cuda = device.type == "cuda"
    if cuda: yolo.get_gpu_prop(show=True)
    print("\ndevice: {}".format(device))
    
    args.amp = False
    if cuda and torch.__version__ >= "1.6.0":
        capability = torch.cuda.get_device_capability()[0]
        if capability >= 7: # 7 refers to RTX series GPUs
            args.amp = True
            print("Automatic mixed precision (AMP) is enabled!")
            
    # ---------------------- prepare data loader ------------------------------- #
    
    DALI = cuda & yolo.DALI & (args.dataset == "coco")
    
    if DALI:
        print("Nvidia DALI is utilized!")
        d_test = yolo.DALICOCODataLoader(
            args.file_root, args.ann_file, args.batch_size, collate_fn=yolo.collate_wrapper)
    else:
        dataset_test = yolo.datasets(args.dataset, args.file_root, args.ann_file, train=True) # set train=True for eval
        sampler_test = torch.utils.data.SequentialSampler(dataset_test)

        batch_sampler_test = yolo.GroupedBatchSampler(
            sampler_test, dataset_test.aspect_ratios, args.batch_size)
        
        args.num_workers = min(os.cpu_count() // 2, 8, args.batch_size if args.batch_size > 1 else 0)
        data_loader_test = torch.utils.data.DataLoader(
            dataset_test, batch_sampler=batch_sampler_test, num_workers=args.num_workers,  
            collate_fn=yolo.collate_wrapper, pin_memory=cuda)

        d_test = yolo.DataPrefetcher(data_loader_test) if cuda else data_loader_test
    
    # -------------------------------------------------------------------------- #

    yolo.setup_seed(3)
    
    model_sizes = {"small": (0.33, 0.5), "medium": (0.67, 0.75), "large": (1, 1), "extreme": (1.33, 1.25)}
    num_classes = len(d_test.dataset.classes)
    model = yolo.YOLOv5(num_classes, model_sizes[args.model_size], **args.kwargs).to(device)
    model.head.eval_with_loss = args.eval_with_loss
    
    checkpoint = torch.load(args.ckpt_path, map_location=device)
    if "ema" in checkpoint:
        model.load_state_dict(checkpoint["ema"][0])
        print(checkpoint["eval_info"])
    else:
        model.load_state_dict(checkpoint)

    model.fuse()
    print("evaluating...")
    B = time.time()
    eval_output, iter_eval = yolo.evaluate(model, d_test, device, args, evaluation=args.evaluation)
    B = time.time() - B
    print(eval_output)
    print("\ntotal time of this evaluation: {:.2f} s, speed: {:.2f} FPS".format(B, args.batch_size / iter_eval))
    


# Resultados

## Mask

### Modelo mask small

In [3]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args([]) # [] is needed when using Jupyter Notebook.
    
    args.use_cuda = True
    
    args.dataset = "coco"
    args.file_root = "data/coco2017/val2017"
    args.ann_file = "data/coco2017/annotations/instances_val2017.json"
    args.ckpt_path = "Varmedian/mask-s-200.pth"
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.json")
    
    args.batch_size = 32
    args.iters = -1
    
    args.model_size = "small"
    args.kwargs = {"img_sizes": 420, "score_thresh": 0.1, "detections": 100} # mAP 34.6 FPS 451
    #args.kwargs = {"img_sizes": 672, "score_thresh": 0.001, "detections": 300} # mAP 36.1. take more(2x-4x) time in total
    args.evaluation = True
    args.eval_with_loss = False
    
    main(args)
    
    

cuda: True
available GPU(s): 1
0: {'name': 'NVIDIA GeForce RTX 3060 Ti', 'capability': [8, 6], 'total_momory': 8.0, 'sm_count': 38}

device: cuda
Automatic mixed precision (AMP) is enabled!
loading annotations into memory...
Done (t=0.45s)
creating index...
index created!
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.209
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.332
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.225
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.061
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.230
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.327
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.190
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.260
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 

### Modelo mask medium

In [4]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args([]) # [] is needed when using Jupyter Notebook.
    
    args.use_cuda = True
    
    args.dataset = "coco"
    args.file_root = "data/coco2017/val2017"
    args.ann_file = "data/coco2017/annotations/instances_val2017.json"
    args.ckpt_path = "Varmedian/mask-m-200.pth"
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.json")
    
    args.batch_size = 32
    args.iters = -1
    
    args.model_size = "medium"
    args.kwargs = {"img_sizes": 420, "score_thresh": 0.1, "detections": 100} # mAP 34.6 FPS 451
    #args.kwargs = {"img_sizes": 672, "score_thresh": 0.001, "detections": 300} # mAP 36.1. take more(2x-4x) time in total
    args.evaluation = True
    args.eval_with_loss = False
    
    main(args)
    
    

cuda: True
available GPU(s): 1
0: {'name': 'NVIDIA GeForce RTX 3060 Ti', 'capability': [8, 6], 'total_momory': 8.0, 'sm_count': 38}

device: cuda
Automatic mixed precision (AMP) is enabled!
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.270
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.398
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.292
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.232
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.323
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 

## VAR

### Modelo 10 fine small

In [5]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args([]) # [] is needed when using Jupyter Notebook.
    
    args.use_cuda = True
    
    args.dataset = "coco"
    args.file_root = "data/coco2017/val2017"
    args.ann_file = "data/coco2017/annotations/instances_val2017.json"
    args.ckpt_path = "Varmedian/var-s-210.pth"
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.json")
    
    args.batch_size = 32
    args.iters = -1
    
    args.model_size = "small"
    args.kwargs = {"img_sizes": 420, "score_thresh": 0.1, "detections": 100} # mAP 34.6 FPS 451
    #args.kwargs = {"img_sizes": 672, "score_thresh": 0.001, "detections": 300} # mAP 36.1. take more(2x-4x) time in total
    args.evaluation = True
    args.eval_with_loss = False
    
    main(args)
    
    

cuda: True
available GPU(s): 1
0: {'name': 'NVIDIA GeForce RTX 3060 Ti', 'capability': [8, 6], 'total_momory': 8.0, 'sm_count': 38}

device: cuda
Automatic mixed precision (AMP) is enabled!
loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.314
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.218
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.316
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.181
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.244
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 

In [6]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args([]) # [] is needed when using Jupyter Notebook.
    
    args.use_cuda = True
    
    args.dataset = "coco"
    args.file_root = "data/coco2017/val2017"
    args.ann_file = "data/coco2017/annotations/instances_val2017.json"
    args.ckpt_path = "Varmedian/varmod-s-210.pth"
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.json")
    
    args.batch_size = 32
    args.iters = -1
    
    args.model_size = "small"
    args.kwargs = {"img_sizes": 420, "score_thresh": 0.1, "detections": 100} # mAP 34.6 FPS 451
    #args.kwargs = {"img_sizes": 672, "score_thresh": 0.001, "detections": 300} # mAP 36.1. take more(2x-4x) time in total
    args.evaluation = True
    args.eval_with_loss = False
    
    main(args)
    
    

cuda: True
available GPU(s): 1
0: {'name': 'NVIDIA GeForce RTX 3060 Ti', 'capability': [8, 6], 'total_momory': 8.0, 'sm_count': 38}

device: cuda
Automatic mixed precision (AMP) is enabled!
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.314
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.223
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.313
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.181
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.245
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 

### Modelo 10 fine medium

In [7]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args([]) # [] is needed when using Jupyter Notebook.
    
    args.use_cuda = True
    
    args.dataset = "coco"
    args.file_root = "data/coco2017/val2017"
    args.ann_file = "data/coco2017/annotations/instances_val2017.json"
    args.ckpt_path = "Varmedian/var-m-210.pth"
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.json")
    
    args.batch_size = 32
    args.iters = -1
    
    args.model_size = "medium"
    args.kwargs = {"img_sizes": 420, "score_thresh": 0.1, "detections": 100} # mAP 34.6 FPS 451
    #args.kwargs = {"img_sizes": 672, "score_thresh": 0.001, "detections": 300} # mAP 36.1. take more(2x-4x) time in total
    args.evaluation = True
    args.eval_with_loss = False
    
    main(args)
    
    

cuda: True
available GPU(s): 1
0: {'name': 'NVIDIA GeForce RTX 3060 Ti', 'capability': [8, 6], 'total_momory': 8.0, 'sm_count': 38}

device: cuda
Automatic mixed precision (AMP) is enabled!
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.240
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.261
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.269
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.378
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.208
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.282
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 

In [8]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args([]) # [] is needed when using Jupyter Notebook.
    
    args.use_cuda = True
    
    args.dataset = "coco"
    args.file_root = "data/coco2017/val2017"
    args.ann_file = "data/coco2017/annotations/instances_val2017.json"
    args.ckpt_path = "Varmedian/varmod-m-210.pth"
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "results.json")
    
    args.batch_size = 32
    args.iters = -1
    
    args.model_size = "medium"
    args.kwargs = {"img_sizes": 420, "score_thresh": 0.1, "detections": 100} # mAP 34.6 FPS 451
    #args.kwargs = {"img_sizes": 672, "score_thresh": 0.001, "detections": 300} # mAP 36.1. take more(2x-4x) time in total
    args.evaluation = True
    args.eval_with_loss = False
    
    main(args)
    
    

cuda: True
available GPU(s): 1
0: {'name': 'NVIDIA GeForce RTX 3060 Ti', 'capability': [8, 6], 'total_momory': 8.0, 'sm_count': 38}

device: cuda
Automatic mixed precision (AMP) is enabled!
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.358
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.275
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.382
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.210
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.287
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 